In [1]:
import requests
import pandas as pd
import numpy as np
import re

In [2]:
data = pd.read_excel('ks-projects-201612.xlsx')
dataks = data.copy()
dataks.shape

(323750, 17)

In [3]:
dataks = dataks.drop(['Unnamed: 13','Unnamed: 14','Unnamed: 15','Unnamed: 16'],axis=1)

In [4]:
dataks.columns = ['ID', 'name', 'category', 'main_category', 'currency', 'deadline',
       'goal', 'launched', 'pledged', 'state', 'backers', 'country',
       'usd_pledged']

In [5]:
maincat = dict(dataks.main_category.value_counts())
main = [e for e in maincat if maincat[e]>100]
dataks.main_category = dataks.main_category.apply(lambda x : x if x in main else 'Other')

In [6]:
null_cols = dataks.isnull().sum()
null_cols[null_cols>0]

name             44
category          5
usd_pledged    3790
dtype: int64

In [7]:
null_pledged = dataks[(dataks['usd_pledged'].isnull()==True)]
def uspled():
    for w in null_pledged.index:
        if null_pledged.loc[w,'currency'] == 'USD':
            dataks.loc[w,'usd_pledged'] = dataks.loc[w,'pledged']
uspled()
dataks.dropna(subset=['usd_pledged'],how='all',inplace=True)

In [8]:
dataks.dropna(subset=['category'],how='all',inplace=True)

In [10]:
dataks['name'] = dataks['name'].fillna('NA')

In [11]:
dataks = dataks.drop(['category'],axis=1)

In [12]:
dataks.shape

(322688, 12)

In [14]:
curr = ['USD','GBP','CAD','EUR','AUD','SEK','NZD','DKK','NOK','CHF','MXN','SGD','HKD']

In [15]:
dataks.currency = dataks.currency.apply(lambda x : x if x in curr else 'Other')

In [16]:
othercurr = list(dataks[(dataks['currency']=='Other')].index)
dataks = dataks.drop(othercurr)

In [17]:
othercount = list(dataks[(dataks['country']=='N,"0')].index)
dataks = dataks.drop(othercount)

In [20]:
dataks.shape

(319328, 12)

In [34]:
dataks[['goal','pledged','backers','usd_pledged']] = dataks[['goal','pledged','backers','usd_pledged']].astype(float)

In [35]:
ks = pd.DataFrame(dataks)
ks

,ID,name,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged
0,1000002330,The Songs of Adelaide & Abullah,Publishing,GBP,2015-10-09 11:36:00,1000.0,2015-08-11 12:12:28,0.0,failed,0.0,GB,0.0
1,1000004038,Where is Hank?,Film & Video,USD,2013-02-26 00:20:50,45000.0,2013-01-12 00:20:50,220.0,failed,3.0,US,220.0
2,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,USD,2012-04-16 04:24:11,5000.0,2012-03-17 03:24:11,1.0,failed,1.0,US,1.0
3,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,USD,2015-08-29 01:00:00,19500.0,2015-07-04 08:35:03,1283.0,canceled,14.0,US,1283.0
4,1000014025,Monarch Espresso Bar,Food,USD,2016-04-01 13:38:27,50000.0,2016-02-26 13:38:27,52375.0,successful,224.0,US,52375.0
...,...,...,...,...,...,...,...,...,...,...,...,...
323745,999976400,ChknTruk Nationwide Charity Drive 2014 (Canceled),Film & Video,USD,2014-10-17 02:35:30,50000.0,2014-09-17 02:35:30,25.0,canceled,1.0,US,25.0
323746,999977640,The Tribe,Film & Video,USD,2011-07-19 03:35:14,1500.0,2011-06-22 03:35:14,155.0,failed,5.0,US,155.0
323747,999986353,Walls of Remedy- New lesbian Romantic Comedy f...,Film & Video,USD,2010-08-16 05:59:00,15000.0,2010-07-01 19:40:30,20.0,failed,1.0,US,20.0
323748,999987933,BioDefense Education Kit,Technology,USD,2016-02-13 02:00:00,15000.0,2016-01-13 18:13:53,200.0,failed,6.0,US,200.0


In [72]:
kscross = pd.crosstab(ks.main_category,ks.state)
kscross['Succ_Rate']=kscross['successful']/(kscross['canceled']+kscross['failed']+kscross['live']+kscross['successful']+kscross['suspended'])*100
kscross.sort_values('successful',ascending=False)

state,canceled,failed,live,successful,suspended,Succ_Rate
main_category,,,,,,
Music,2934,19137,467,21671,125,48.881220
Film & Video,5151,29641,500,21393,92,37.678990
Publishing,3084,19888,448,10255,51,30.406808
Art,1885,12092,256,9654,72,40.293835
Games,4964,13013,462,9385,176,33.517857
Design,3119,12039,549,7958,197,33.350096
Theater,547,3337,66,5990,21,60.134525
Food,1931,13602,278,5256,128,24.798301
Technology,3749,16344,632,5061,323,19.384120


In [39]:
ks.describe()

,ID,goal,pledged,backers,usd_pledged
count,3.193280e+05,3.193280e+05,3.193280e+05,319328.000000,3.193280e+05
mean,1.074818e+09,4.764788e+04,8.764063e+03,102.841151,7.847678e+03
std,6.193753e+08,1.146304e+06,8.998115e+04,940.383098,8.468405e+04
min,5.971000e+03,1.000000e-02,0.000000e+00,0.000000,0.000000e+00
25%,5.373998e+08,2.000000e+03,3.000000e+01,2.000000,2.500000e+01
50%,1.075808e+09,5.000000e+03,6.100000e+02,12.000000,5.350000e+02
75%,1.611033e+09,1.500000e+04,3.939450e+03,56.000000,3.575000e+03
max,2.147476e+09,1.000000e+08,2.033899e+07,219382.000000,2.033899e+07


In [60]:
pais=ks.groupby('country', as_index=True).agg({'usd_pledged':'sum'})
pais.sort_values('usd_pledged',ascending=False)

,usd_pledged
country,
US,2.152237e+09
GB,1.727513e+08
CA,6.632724e+07
AU,2.789101e+07
DE,1.901213e+07
FR,1.833704e+07
NL,1.100838e+07
IT,6.832740e+06
SE,5.257782e+06
